#Data Analyst Interview Notebook

## Introduction

This notebook is designed to simulate real-world data that you could see from our technology.

It is designed to test your proficiency with Python and Pandas, and your ability to find and present insights.

This data is confidential; we expect you to not share this data with anyone. And we expect you to not get any external assistance.



---


You are working with a US retail customer on a pilot deployment.  They are using technology to track their merchandise throughout their supply chain.  The flow of their supply is:

*   **DC 1:**  Orders are filled and palletized.
*   **Truck:** Pallets travel from the DC 1 to DC 2 via semi-truck.
*   **DC 2:**  Pallets are unloaded, and additional merchandise may be added.  They are then reloaded onto a new truck.
*   **Truck:** Pallets travel from DC 2 to the Store.
*   **Store:** Pallets are unloaded, cases are removed, and stocked, and the empty cases are left behind the building awaiting pickup.

Your job is to dig into the data and find compelling insights to show the value fo the technology and help move the contract from a pilot into a full scaled deployment.



---

Each section has defined questions along with point values to help you prioritize your time.  

In addition to the defined questions, the overall notebook and code presentation will be evaluated, and you will be awarded up to 10 points based on the clarity, cleaniness, and efficiency of the code.


## Part 0: Imports

Import necessary packages and

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly as p
from datetime import datetime, date
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon


### Dataset Overview

* Site:  A large space that could contain multiple readers. Ex: DC 1.
* Zone:  Point of interest. These represent areas in which repeaters are installed. These can be thought of as sub-zones.  Ex: Dock Doors.
* Asset ID: The unique ID of the asset.
* Asset Type: The type of thing that is detected (ie tote).
* Device ID: The unique gateway reader ID that detected the device in the zone (there can be multiple in one zone).
* Time est: The time in EST.
* Lon: Longituge
* Lat: Latitude
* Temperature_C / F: Temperature in Celsius, Fahrentheit

## PART 1: Data Overview

### Question 1:

* How many unique cases were we tracking throughout this pilot. (1 pt)
* What are the unique zones we could see (1 pt)
* How many POI's are in each Zone. (2 pts)


In [16]:
# YOUR CODE HERE:
df = pd.read_excel('tote_shipment.xlsx')
df.head()
#define columns names
df.columns = ['Tipo', 'Asset ID', 'Tag ID', 'Sitio', 'Zona', 'Device ID', 'Tiempo', 'lat', 'lng', 'Temp_C', 'Temp_F']
df





,Tipo,Asset ID,Tag ID,Sitio,Zona,Device ID,Tiempo,lat,lng,Temp_C,Temp_F
0,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:59:26.628,47.79158,-65.68902,23.0,73.40
1,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,EC5B0499234F,2022-08-02 12:00:22.660,47.79158,-65.68902,23.0,73.40
2,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 12:01:11.234,47.79158,-65.68902,23.0,73.40
3,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,3D8B2BDB8673,2022-08-02 13:47:58.172,47.79158,-65.68902,23.5,74.30
4,tote,2,(01)00850027865010(21)0082T0670,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:58:55.049,47.79158,-65.68902,23.0,73.40
...,...,...,...,...,...,...,...,...,...,...,...
130056,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:48:51.273,47.74923,-65.64101,23.1,73.58
130057,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:49:58.699,47.74923,-65.64101,23.5,74.30
130058,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:52:45.173,47.74923,-65.64101,23.0,73.40
130059,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:53:48.228,47.74923,-65.64101,23.0,73.40


In [3]:
#How many unique cases were we tracking throughout this pilot. (1 pt)
df_u=df.describe(include="all")
print(df_u.iloc[1])


Tipo              1
Asset ID        NaN
Tag ID           34
Sitio             4
Zona             15
Device ID        38
Tiempo       129887
lat             NaN
lng             NaN
Temp_C          NaN
Temp_F          NaN
Name: unique, dtype: object


<ipython-input-3-56a3a0edf219>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_u=df.describe(include="all")


In [4]:
#What are the unique zones we could see (1 pt)
df_uz=df["Zona"].unique()
print(f"The unique zones are: {df_uz}")

The unique zones are: ['dock_doors_DC1' 'dock_doors_DC2' 'Forklift3_DC1' 'pallet_assembly_DC1'
 'PhoneKit1Bridge' 'PhoneKit2Bridge' 'PhoneKit2GW' 'point_of_sale_Store'
 'receiving_Store' 'staging_DC1' 'staging_DC2' 'store_back_Store'
 'store_front_Store' 'Forklift1_DC1' 'PhoneKit1GW']


In [5]:
#How many POI's are in each Zone. (2 pts)
poi=df["Zona"].nunique()
print(f"There are {poi} POI's in each zone")

There are 15 POI's in each zone


### Question 2:

* What is the temperature range we see?  (1pt)
* Where is temperature the highest and lowest (1pt)

In [6]:
# YOUR CODE HERE:
# What is the temperature range we see? (1pt)
range=df['Temp_C'].max() - df['Temp_C'].min()
print(f"The range is: {range}")

The range is: 25.0


In [7]:
#Where is temperature the highest and lowest (1pt)
temp_min=df['Temp_C'].max()
temp_max=df['Temp_C'].min()
print(f"Highest temperature is: {temp_max} and lowest is: {temp_min}")

Highest temperature is: 19.0 and lowest is: 44.0


## Part 2: The Journey of a Case

### Question 3:

* Create a visualization that shows where a case was at over time at the zone or POI level. Imagine that this would be included in your presentation to the customer. (Non techical audience) (3 pts)

In [8]:
# YOUR CODE HERE:


fig = px.scatter(df, x = 'Zona', y = 'Tiempo')

fig

Output hidden; open in https://colab.research.google.com to view.

### Question 4:

* Visualize how the temperatue changes over time along its journey.  Imagine that this would be included in your presentation to the customer. (Non techical audience) (4 pts)



In [15]:
# YOUR CODE HERE:
px.scatter(df, x = 'Tiempo', y = 'Zona', color = 'Temp_C')

Output hidden; open in https://colab.research.google.com to view.

### Question 5:
* Visualize the lon lat data on a map to show how the case traveled.  You may incorporate any other additional information to make this more impactful. Imagine that this would be included in your presentation to the customer. (Non techical audience) (5 pts)

**Do not worry if this looks like non-sense on a map.  Ex:  The trip may appear to occur over water or in a forest because this is a toy dataset.**

In [ ]:
# YOUR CODE HERE:
#we search for the map of the area (Baie des Chaleurs), read in the shape-file with GeoPandas and use matplotlib to plot it
street_map = gpd.read_file("Zostera_Zostere.shp")
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax)

In [ ]:
crs = {'init':'epsg:4326'}
df.head()

In [ ]:
#Now we create Points and create a data frame.
geometry = [Point(xy) for xy in zip(df['lng'], df['lat'])]
geo_df = gpd.GeoDataFrame(df, #specify our data
                          crs=crs #specify our coordinate reference system
                          geometry=geometry) #specify the geometry list we created
geo_df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax, alpha=0.4, color='grey')
geo_df[geo_df['WnvPresent'] == 0].plot(ax=ax,
                                       markersize=20,
                                       color='blue',
                                       marker='o',
                                       label='Neg')
geo_df[geo_df['WnvPresent'] == 1].plot(ax=ax,
                                       markersize=20,
                                       color='red',
                                       marker='^',
                                       label='Pos')
plt.legend(prop={'size':15})

# Part 3: Customer Questions


### Question 6:

The customer wants to understand the efficieny of ther DC operations.
* Based on what you see in the data, (all zones except for STORE), which parts of their operation are most & least "efficient? (10 pts)

In [30]:
# YOUR CODE HERE
# filter the Sites without store
df_eff= df[(df['Sitio'] != 'Store')]

df_eff.sort_values(by='Tiempo')
df_eff.head()



,Tipo,Asset ID,Tag ID,Sitio,Zona,Device ID,Tiempo,lat,lng,Temp_C,Temp_F
0,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:59:26.628,47.79158,-65.68902,23.0,73.4
1,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,EC5B0499234F,2022-08-02 12:00:22.660,47.79158,-65.68902,23.0,73.4
2,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 12:01:11.234,47.79158,-65.68902,23.0,73.4
3,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,3D8B2BDB8673,2022-08-02 13:47:58.172,47.79158,-65.68902,23.5,74.3
4,tote,2,(01)00850027865010(21)0082T0670,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:58:55.049,47.79158,-65.68902,23.0,73.4


In [31]:
df_eff.tail()

,Tipo,Asset ID,Tag ID,Sitio,Zona,Device ID,Tiempo,lat,lng,Temp_C,Temp_F
126903,tote,205,(01)00850027865010(21)0082T1136,DC 2,staging_DC2,BAE5E8CF978C,2022-08-04 01:50:59.214,47.79423,-65.75888,25.0,77.0
126904,tote,205,(01)00850027865010(21)0082T1136,DC 2,staging_DC2,BAE5E8CF978C,2022-08-04 02:04:28.220,47.79366,-65.75921,25.0,77.0
126905,tote,205,(01)00850027865010(21)0082T1136,DC 2,staging_DC2,BAE5E8CF978C,2022-08-04 03:09:53.268,47.79366,-65.75921,25.0,77.0
126906,tote,205,(01)00850027865010(21)0082T1136,DC 2,staging_DC2,BAE5E8CF978C,2022-08-04 03:11:19.472,47.79366,-65.75921,25.0,77.0
126907,tote,205,(01)00850027865010(21)0082T1136,DC 2,staging_DC2,BAE5E8CF978C,2022-08-04 03:29:59.538,47.79366,-65.75921,25.0,77.0


YOUR TEXT ANSWER HERE

### Question 7:

The customer wants to understand the stocking efficiency in stores.
* Based on what you see in the data, how quickly did the store unload and stock the merchandise. (5 pts)
* How could this be converted in a KPI that a regional manager could track?  (5 pts)

In [32]:
# YOUR CODE HERE
# Based on what you see in the data, how quickly did the store unload and stock the merchandise. (5 pts
# we take the average  of tiempo
avg = df['Tiempo'].mean()
print(f'the average is: {avg}')



the average is: 2022-08-04 14:15:47.732850176


In [20]:
# How could this be converted in a KPI that a regional manager could track? (5 pts)


YOUR TEXT ANSWER HERE

### Question 8:

Please explain what you would ask for and what you will do with this data, given that you can talk with the following people (no code needed):


YOUR TEXT ANSWER HERE
* a. X
* b. Y

## Part 4: Bonus Insights

### Question 8

The customer is open to hearing about additional insights you found in the data above and beyond what they asked for.
* Based on what you can see in the data, are there any other interesting insights that the customer may want to hear about? (Up to 15 bonus points)



In [ ]:
# YOUR CODE HERE
# Another metrics that could be interesting would be temperature by time

YOUR TEXT ANSWER HERE
